# Importing Libraries

In [1]:
import pandas as pd
import folium
import requests
import json

In [2]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Popular Spots.csv')
df

,Postal Code,Borough,Spot Name,Latitude,Longitude
0,M5V,Downtown Toronto,CN Tower,43.628947,-79.394420
1,M5S,Downtown Toronto,Royal Ontario Museum,43.662696,-79.400049
2,M5T,Downtown Toronto,Art Gallery of Ontario,43.653206,-79.400049
3,M5J,Downtown Toronto,Scotiabank Arena,43.640816,-79.381752
4,M1B,Scarborough,Toronto Zoo,43.806686,-79.194353
5,M5R,Central Toronto,Casa Loma,43.672710,-79.405678
6,M5E,Downtown Toronto,St. Lawrence Market,43.644771,-79.373306
7,M5H,Downtown Toronto,Toronto City Hall,43.650571,-79.384568
8,M5B,Downtown Toronto,CF Toronto Eaton Centre,43.657162,-79.378937
9,M5A,Downtown Toronto,The Distillery Historic District,43.654260,-79.360636


Setting the map view over Toronto City and marking our popular spots

In [3]:
latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, spot in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Spot Name']):
    label = '{}, {}'.format(spot, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Configuring the Foursquare Account Credentials

In [4]:
CLIENT_ID = 'OLEOCF2MYTD4TFBAL5FNZELSX14JMNJW30ZPWCZP52QUTTM3' # Foursquare ID
CLIENT_SECRET = 'G1Y5T5JD5ZB1KHSXSUMCYZQPFE0MPTES0RLUMO4O3QME4STH' # Foursquare Secret
VERSION = '20200721' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # radius in Metres(1000m = 1 km)
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OLEOCF2MYTD4TFBAL5FNZELSX14JMNJW30ZPWCZP52QUTTM3
CLIENT_SECRET:G1Y5T5JD5ZB1KHSXSUMCYZQPFE0MPTES0RLUMO4O3QME4STH


Defining a Method to get nearby venues

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Spot Name', 
                  'Spot Name Latitude', 
                  'Spot Name Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Calling the function defined above

In [7]:
toronto_venues = getNearbyVenues(names=df['Spot Name'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

CN Tower
Royal Ontario Museum
Art Gallery of Ontario
Scotiabank Arena
Toronto Zoo
Casa Loma
St. Lawrence Market
Toronto City Hall
CF Toronto Eaton Centre
The Distillery Historic District
High Park
Ontario Science Centre
Scarborough Bluffs
Yorkdale Shopping Centre
Woodbine Beach


Printing the shape of the result of the above query

In [8]:
print(toronto_venues.shape)
toronto_venues.head()

(611, 7)


,Spot Name,Spot Name Latitude,Spot Name Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,CN Tower,43.628947,-79.39442,Billy Bishop Toronto City Airport (YTZ) (Billy...,43.631683,-79.396033,Airport
1,CN Tower,43.628947,-79.39442,Porter Lounge,43.630680,-79.395756,Airport Lounge
2,CN Tower,43.628947,-79.39442,Toronto Harbour,43.633045,-79.396484,Harbor / Marina
3,CN Tower,43.628947,-79.39442,Billy Bishop Café,43.631132,-79.396139,Airport Food Court
4,CN Tower,43.628947,-79.39442,Air Canada Check-In Counter,43.631226,-79.395987,Airport Terminal


Grouping by Neighbourhood

In [9]:
toronto_venues.groupby('Spot Name').count()

,Spot Name Latitude,Spot Name Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Spot Name,,,,,,
Art Gallery of Ontario,66,66,66,66,66,66
CF Toronto Eaton Centre,100,100,100,100,100,100
CN Tower,17,17,17,17,17,17
Casa Loma,22,22,22,22,22,22
High Park,14,14,14,14,14,14
Ontario Science Centre,21,21,21,21,21,21
Royal Ontario Museum,35,35,35,35,35,35
Scarborough Bluffs,3,3,3,3,3,3
Scotiabank Arena,100,100,100,100,100,100


Printing the unique neighbourhoods

In [10]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 181 uniques categories.


One Hot Encoding the result

In [11]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Spot Name'] = toronto_venues['Spot Name'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Spot Name,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,...,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,CN Tower,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,CN Tower,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,CN Tower,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,CN Tower,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,CN Tower,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
toronto_grouped = toronto_onehot.groupby('Spot Name').mean().reset_index()
toronto_grouped

,Spot Name,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,...,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Art Gallery of Ontario,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.015152,0.000000,0.000000,0.000000,0.015152,0.000000,0.045455,0.00,0.000000,0.000000,0.015152,0.015152,0.000000,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.015152,0.000000,0.000000,0.00,0.030303,0.000000,0.075758,...,0.015152,0.000000,0.00,0.015152,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.015152,0.015152,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.015152,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.060606,0.000000,0.045455,0.015152,0.00,0.000000
1,CF Toronto Eaton Centre,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.000000,0.000000,0.030000,0.00,0.010000,0.010000,0.030000,...,0.000000,0.010000,0.02,0.000000,0.000000,0.010000,0.00,0.00,0.00,0.010000,0.00,0.000000,0.010000,0.010000,0.010000,0.000000,0.01,0.000000,0.000000,0.00,0.010000,0.00,0.010000,0.00,0.010000,0.000000,0.000000,0.000000,0.01,0.02,0.010000,0.020000,0.01,0.00,0.000000,0.010000,0.010000,0.010000,0.00,0.000000
2,CN Tower,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.058824,0.000000,0.00,0.00,0.00,0.000000,0.00,0.058824,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Casa Loma,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.045455,0.000000,0.136364,...,0.000000,0.045455,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.136364,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.045455,0.000000,0.000000,0.000000,0.00,0.000000
4,High Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.00,0.142857,0.000000,0.000000,0

# Printing the first 5 type of venues nearby each spot
**Higher the freq column value, the more common the venue is around our desired location**

In [13]:
num_top_venues = 5

for hood in toronto_grouped['Spot Name']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Spot Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Art Gallery of Ontario----
                           venue  freq
0                           Café  0.08
1                    Coffee Shop  0.06
2  Vegetarian / Vegan Restaurant  0.06
3          Vietnamese Restaurant  0.05
4                            Bar  0.05


----CF Toronto Eaton Centre----
             venue  freq
0   Clothing Store  0.10
1      Coffee Shop  0.09
2   Cosmetics Shop  0.03
3             Café  0.03
4  Bubble Tea Shop  0.03


----CN Tower----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4       Coffee Shop  0.06


----Casa Loma----
            venue  freq
0            Café  0.14
1  Sandwich Place  0.14
2     Coffee Shop  0.09
3   Metro Station  0.05
4       BBQ Joint  0.05


----High Park----
                         venue  freq
0                    Gift Shop  0.14
1               Breakfast Spot  0.14
2  Eastern European Restaurant  0.07
3                    Bookstore  0.07
4      

**Sorting the venue popularity in Descending order**

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Spot Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
spot_venues_sorted = pd.DataFrame(columns=columns)
spot_venues_sorted['Spot Name'] = toronto_grouped['Spot Name']

for ind in np.arange(toronto_grouped.shape[0]):
    spot_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

spot_venues_sorted

,Spot Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Art Gallery of Ontario,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Mexican Restaurant,Vietnamese Restaurant,Bar,Burger Joint,Dessert Shop,Pizza Place,Dumpling Restaurant
1,CF Toronto Eaton Centre,Clothing Store,Coffee Shop,Italian Restaurant,Bubble Tea Shop,Hotel,Cosmetics Shop,Café,Japanese Restaurant,Bookstore,Lingerie Store
2,CN Tower,Airport Service,Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Coffee Shop
3,Casa Loma,Sandwich Place,Café,Coffee Shop,BBQ Joint,Burger Joint,Liquor Store,Martial Arts Dojo,Metro Station,Middle Eastern Restaurant,Indian Restaurant
4,High Park,Gift Shop,Breakfast Spot,Italian Restaurant,Movie Theater,Cuban Restaurant,Restaurant,Bookstore,Dessert Shop,Eastern European Restaurant,Bar
5,Ontario Science Centre,Coffee Shop,Beer Store,Restaurant,Gym,Asian Restaurant,Supermarket,Bike Shop,Dim Sum Restaurant,Japanese Restaurant,Italian Restaurant
6,Royal Ontario Museum,Café,Bookstore,Bar,Japanese Restaurant,Sandwich Place,Restaurant,Bakery,Yoga Studio,Italian Restaurant,Beer Bar
7,Scarborough Bluffs,American Restaurant,Skating Rink,Motel,Yoga Studio,Cuban Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
8,Scotiabank Arena,Coffee Shop,Aquarium,Café,Hotel,Fried Chicken Joint,Brewery,Scenic Lookout,Sporting Goods Shop,Restaurant,Bar
9,St. Lawrence Market,Coffee Shop,Cheese Shop,Seafood Restaurant,Bakery,Cocktail Bar,Café,Pharmacy,Farmers Market,Beer Bar,Restaurant


As per the above analysis of the data, we can see the top 10 locations around the popular tourist spots that we have chosen. If someone was planning to open a Coffee Shop for example, then opening one around Art Gallery or Ontario would not be such a good idea.

Since there are already so many Coffee Shops and Cafes present in the area, setting your business up in the Area will be difficult as you will have a lot of competing stores.

On the other hand, setting up a Coffee Shop around Toronto Zoo, Woodbine Beach, Scarborough Bluffs and High Park would be ideal, as Coffee shops are less frequently spotted in the area. 

Thus it is an ideal spot for a new business to flourish.